<a href="https://colab.research.google.com/github/DouglasDiasEng/Action-Recognition-/blob/main/Guide_Step_by_Step_(English).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing the Libraries

To get started, we install two essntial libraries: **OpenCV** and **TensorFlow Hub**, which will be used to process the videos and load the pre-trained action recognition model.




In [1]:
!pip install -q opencv-python tensorflow_hub

## Importing the Libraries

In this section, we import all the necessary libraries for video processing, model loading, temporary file management, and displaying results in the notebook. The print statement shows the current version of TensorFlow installed in the environment, ensuring it is compatible with the model bing loaded.



In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from google.colab.patches import cv2_imshow
import re
import os
import tempfile
import ssl
import cv2
import numpy as np
from urllib import request
from IPython.display import HTML
from base64 import b64encode

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.19.0


## Loading the Kinetics-400 Lables

The I3D model uses the action list from the **Kinetics-400** dataset, which contains 400 different classes.
In this section, we download this list directly from the official repository and load it into a Python list.



In [3]:
url_classes = 'https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt'
with request.urlopen(url_classes) as file:
  label = [row.decode('utf-8').strip() for row in file.readlines()]

print("Total number of classes: ", len(label))
print("Some classes: ", label[:10])

Total number of classes:  400
Some classes:  ['abseiling', 'air drumming', 'answering questions', 'applauding', 'applying cream', 'archery', 'arm wrestling', 'arranging flowers', 'assembling computer', 'auctioning']


## Initial Setup for Downloading Videos

Before processing any video, we need to define where they will be fetched from and where they will by temporarily stored in the environmnt.
In this section, we create these configurations and define a function to list know videos.

In [4]:
root_folder = 'https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/'
context = ssl._create_unverified_context()
cache_videos = tempfile.mkdtemp()
print("Temporary folder for videos:", cache_videos)

def list_videos():
  known_videos = [
    'v_PizzaTossing_g23_c03.avi',
    'v_ApplyEyeMakeup_g01_c01.avi',
    'v_Archery_g01_c01.avi'
]
  return known_videos

print("List of known videos", list_videos())

Temporary folder for videos: /tmp/tmp1923iquq
List of known videos ['v_PizzaTossing_g23_c03.avi', 'v_ApplyEyeMakeup_g01_c01.avi', 'v_Archery_g01_c01.avi']


## Function to Download and Store the Video Locally

This function is responsable for downloading a video from the internet and saving it in a temporary folder.
If the video has already been downloaded beforee, it prevents repeated downloads.


In [5]:
def save_video(video_name):
  cache_path = os.path.join(cache_videos, video_name)
  if not os.path.exists(cache_path):
    url_path = request.urljoin(root_folder, video_name)
    print("Downloading from:", url_path)
    data = request.urlopen(url_path, context=context).read()
    with open(cache_path, 'wb') as f:
      f.write(data)
    print("Video saved in:", cache_path)
  else:
    print("The video already exists in cache:", cache_path)
  return cache_path

video_name = 'v_PizzaTossing_g23_c03.avi'
video_path = save_video(video_name)

print("final path of the video:", video_path)
print("Does the file exist?", os.path.exists(video_path))

Video saved in: /tmp/tmp1923iquq/v_PizzaTossing_g23_c03.avi
final path of the video: /tmp/tmp1923iquq/v_PizzaTossing_g23_c03.avi
Does the file exist? True


## Function to Load the Video with OpenCV

This function reads a video saved on disk, extracts the frames, resizes them to the siz expcted by the model, and converts everything into a NumPy array normalized between 0 and 1.

In [6]:
def load_video(path, visualize=False):
  cap = cv2.VideoCapture(path)
  frames = []

  if not cap.isOpened():
    print("Failed to open the video", path)
    return np.array([])

  while True:
    connected, frame = cap.read()
    if not connected:
      break

    frame = cv2.resize(frame, (224, 224))
    frames.append(frame)

  cap.release()

  if not frames:
    print("No frames were read from the video.")
    return np.array([])

  frames = np.array(frames, dtype=np.float32)

  if visualize and frames.size > 0:
    cv2_imshow(frames[0].astype(np.uint8))

  return frames / 255.0

video_frames = load_video(video_path, visualize=False)
print("Frame array shape", video_frames.shape)

Frame array shape (148, 224, 224, 3)


## Video Conversion and Visualization in the Notebook

This section checks whether the video was loaded correctly and, if so, converts the `.avi` file to `.mp4` and displays the video inside the notebook.



In [ ]:
if video_frames.size > 0:
  output_mp4_path = "/content/output.mp4"
  !ffmpeg -y -i "$video_path" "$output_mp4_path"

  if os.path.exists(output_mp4_path):
    mp4 = open(output_mp4_path, 'rb').read()
    data_url = 'data:video/mp4;base64,' +b64encode(mp4).decode()

    display(HTML(f"""
      <video width="400" controls>
        <source src="{data_url}" type="video/mp4">
      </video>
      """))
  else:
    print("Failed to creat the mp4 file")

## Loading the I3D Model from TensorFlow Hub

In this step, we load the **I3D(inflated 3D ConvNet)** model pre-trained on the **Kinetics-400** dataset directly from **TensorFlow Hub**.
This model is responsible for performing action recognition based on the video frames.



In [8]:
print("Loading model from TensorFlow Hub (this may take a moment)...")
model = hub.load('https://tfhub.dev/deepmind/i3d-kinetics-400/1').signatures['default']
print("Model loaded successfuly!")

Loading model from TensorFlow Hub (this may take a moment)...
Model loaded successfuly!


## Preparing the Video for the Model and Runnig Inference

After loading the video and the model, we need to adjust the frame format to match the input expected by I3D and then perform the action prediction on the video.


In [9]:
if video_frames.size == 0:
  raise ValueError("The video contains no frames. Check the path or the download.")

test_video = tf.constant(video_frames, dtype=tf.float32)[tf.newaxis, ...]
print("Video shape for the model:", test_video.shape)

outputs = model(test_video)
logits = outputs['default'][0]
probabilities = tf.nn.softmax(logits).numpy()




Video shape for the model: (1, 148, 224, 224, 3)


## Analyzing the Inference Results

Now that we have the predicted probabilities for each of the 400 classes in the model, we need to determine which action the model believes is being performed in the video.

In [10]:
top_class = np.argmax(probabilities)
print("Most probable class (index):", top_class)
print("Most likely action:", label[top_class])
print("Confidence: {:.2f}%".format(probabilities[top_class] * 100))
print("\nTop 5 actions:")
top_idx = np.argsort(probabilities)[::-1][:5]
for i in top_idx:
    print(f"{label[i]:35}: {probabilities[i] * 100:5.2f} %")


Most probable class (index): 188
Most likely action: making pizza
Confidence: 36.47%

Top 5 actions:
making pizza                       : 36.47 %
punching bag                       : 28.43 %
catching or throwing frisbee       :  6.93 %
pumping fist                       :  5.17 %
washing dishes                     :  4.70 %
